# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
pd_data.head()

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)

In [4]:
def get_gini(df, label):
    
    gini = 1
    x = df[label].value_counts()
    
    for i in x: 
        p = i / len(df)
        gini -= p**2
    
    return gini

In [5]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)

In [6]:
import itertools # 변수의 모든 클래시 조합을 얻기 위해 itertools 불러오기
def get_binary_split(df, attribute):
    
    tree = list(df[attribute].unique())
    result = []
 
    for i in range(1, len(tree)): 
        a = list(itertools.combinations(tree, i))
        for j in range(len(a)): 
            b = list(a[j])
            result.append(b)
        
    return result

In [7]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기

In [8]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    split = get_binary_split(df, attribute) # feature의 class를 이진분류로 만들기 
    
    for col in split: 
        # 이진분류에 속하는 경우 
        df_in = df[df[attribute].isin(col)]
        df_in_p = len(df_in) / len(df)
        
        gini_in = get_gini(df_in, label) 
        
        
        # 이진분류에 속하지 않는 경우
        df_notin = df[~df[attribute].isin(col)]
        df_notin_p = len(df_notin) / len(df)
        
        gini_notin = get_gini(df_notin, label)
        
        
        # gini index  
        gini = df_in_p * gini_in + df_notin_p * gini_notin
        result[tuple(col)] = gini 
    
    return result

In [9]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split

In [10]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715


## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

In [11]:
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

#### 1. age

In [12]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

In [13]:
age_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
age_min = min(age_dict.keys(), key=(lambda k: age_dict[k]))
print('Min -',age_min, ":", age_dict[age_min])

Min - ('middle_aged',) : 0.35714285714285715


#### 2. income

In [14]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

In [15]:
income_dict = get_attribute_gini_index(pd_data, "income", "class_buys_computer")
income_min = min(income_dict.keys(), key=(lambda k: income_dict[k]))
print('Min -',income_min, ":", income_dict[income_min])

Min - ('high',) : 0.4428571428571429


#### 3. student

In [16]:
get_attribute_gini_index(pd_data, "student", "class_buys_computer")

{('no',): 0.3673469387755103, ('yes',): 0.3673469387755103}

In [17]:
student_dict = get_attribute_gini_index(pd_data, "student", "class_buys_computer")
student_min = min(student_dict.keys(), key=(lambda k: student_dict[k]))
print('Min -',student_min, ":", student_dict[student_min])

Min - ('no',) : 0.3673469387755103


#### 4. credit_rating

In [18]:
get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer")

{('fair',): 0.42857142857142855, ('excellent',): 0.42857142857142855}

In [19]:
credit_rating_dict = get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer")
credit_rating_min = min(credit_rating_dict.keys(), key=(lambda k: credit_rating_dict[k]))
print('Min -',credit_rating_min, ":", credit_rating_dict[credit_rating_min])

Min - ('fair',) : 0.42857142857142855


* 네 가지 변수 중 가장 작은 gini index를 가지는 변수는 **age**이다. 
* **gini index**: Min - ('middle_aged',) = 0.35714285714285715  

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [20]:
# age = middle_aged 일 때 
age_middle = pd_data[pd_data["age"].isin(['middle_aged'])]
age_middle

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [21]:
# age = middle_aged가 아닐 때 
age_middle = pd_data[~pd_data["age"].isin(['middle_aged'])]
age_middle

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [22]:
# 중요한 변수 선정 

def important_feature(df, attribute, label): 
    my_dict = get_attribute_gini_index(df, attribute, label)
    print(attribute, ":", my_dict)
    
    my_dict_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
    print('Min -',my_dict_min, ":", my_dict[my_dict_min])
    print("\n")

In [23]:
columns = ["age", "income", "student", "credit_rating"]

for col in columns: 
    important_feature(age_middle, col, "class_buys_computer")

age : {('youth',): 0.48, ('senior',): 0.48}
Min - ('youth',) : 0.48


income : {('high',): 0.375, ('medium',): 0.48, ('low',): 0.4761904761904763, ('high', 'medium'): 0.4761904761904763, ('high', 'low'): 0.48, ('medium', 'low'): 0.375}
Min - ('high',) : 0.375


student : {('no',): 0.31999999999999984, ('yes',): 0.31999999999999984}
Min - ('no',) : 0.31999999999999984


credit_rating : {('fair',): 0.4166666666666667, ('excellent',): 0.4166666666666667}
Min - ('fair',) : 0.4166666666666667




* 네 가지 변수 중 가장 작은 gini index를 가지는 변수는 **student**이다. 
* 이 때의 **gini index**: Min - ('no',) = 0.31999999999999984

In [24]:
# student = no 일 때 
age_middle_notstudent = age_middle[age_middle["student"].isin(['no'])]
age_middle_notstudent

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
7,youth,medium,no,fair,no
13,senior,medium,no,excellent,no


In [25]:
# student = yes 일 때 
age_middle_student = age_middle[~age_middle["student"].isin(['no'])]
age_middle_student

,age,income,student,credit_rating,class_buys_computer
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes


### 결론
* pd_data는 age=middle_aged일 때 가장 처음으로 나뉘고, 
* student=no일 때 다음으로 나뉜다 


* 따라서 중년층인 경우 컴퓨터를 많이 구매한다고 판단할 수 있고, 
* 중년층이 아닌 경우에는, 학생인 경우에 컴퓨터를 많이 구매한다고 판단할 수 있다. 